In [ ]:

import base64
import pandas as pd
import requests
import seaborn as sns
import spotipy

from dotenv import dotenv_values
from IPython.display import HTML, display
from matplotlib import pyplot
from time import sleep
from scipy.spatial.distance import pdist, squareform
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
config = dotenv_values(".env")

In [ ]:
scope = "user-modify-playback-state app-remote-control streaming user-read-playback-state playlist-modify-public"

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=config["SPOTIFY_CLIENT_ID"],
        client_secret=config["SPOTIFY_CLIENT_SECRET"],
        redirect_uri=config["SPOTIFY_REDIRECT_URL"],
        scope=scope
    )
)


In [ ]:
devices_df = pd.DataFrame(sp.devices()["devices"])
device_id = devices_df[devices_df["name"] == "Tim’s MacBook Pro (2)"]["id"].values[0]
sp_user_id = sp.current_user()["id"]

# Create Playlist
# sp.user_playlist_create(user=sp_user_id, name="Ultimate Xebia Apres Ski", public=True)
playlist_id = "1D5wcDUc6Z9ZlXaItPgrd9"

In [ ]:
def display_two_images(img_url_1, img_url_2: str, text_1: str, text_2: str) -> None:
    """
    """
    html_template = """
        <div class="row">
            <div style="float:left;margin-right:30px;width:300px">{text_1}</div>
            <div style="float:left;margin-right:30px;width:300px">{text_2}</div>
        </div>
        <div style="clear:both"></div>
        <div class="row">
            <img style="float:left;margin-right:30px;" src="data:image/jpeg;base64,{b64_img_1}" width="300" height="300" />
            <img style="float:left;margin-right:30px;" src="data:image/jpeg;base64,{b64_img_2}" width="300" height="300" />
        </div>
    """
    img_bytes_1 = requests.get(img_url_1).content
    img_bytes_2 = requests.get(img_url_2).content
    b64_img_1 =  base64.b64encode(img_bytes_1).decode('ascii')
    b64_img_2 =  base64.b64encode(img_bytes_2).decode('ascii')
    txt = "sjfa;\nakd;f\nsdfa"
    display(
        HTML(
            html_template.format(
                text_1=text_1, text_2=text_2, b64_img_1=b64_img_1,b64_img_2=b64_img_2
            )
        )
    )


In [ ]:
def play_part_of_song(device_id: str, uri: str, start_position_ms: int, n_seconds: int=10) -> None:
    """Start playing a song and pause after duration_seconds"""
    sp.start_playback(
        device_id=device_id,
        uris=[uri],
        position_ms=start_position_ms
    )
    sleep(n_seconds)
    sp.pause_playback(device_id=device_id)

def play_middle_of_track(device_id: str, track_series: pd.Series, n_seconds: int=9) -> None:
    """Play n_seconds on device starting from the middle of the track"""
    play_part_of_song(
        device_id=device_id,
        uri=track_series["uri"],
        start_position_ms=track_series["duration_ms"] / 2,
        n_seconds=n_seconds
    )

def play_audio_feature_extremes(tracks_df, audio_feature, pause_seconds: int=2) -> None:
    """"""
    min_track = tracks_df.sort_values(by=audio_feature, ascending=True).iloc[0]
    max_track = tracks_df.sort_values(by=audio_feature, ascending=False).iloc[0]
    
    display_two_images(
        min_track["album_img_url"],
        max_track["album_img_url"],
        f"Lowest {audio_feature}({min_track[audio_feature]}): <br />{min_track['artist_names']} - {min_track['name']}",
        f"Highest {audio_feature}({max_track[audio_feature]}): <br />{max_track['artist_names']} - {max_track['name']}",
    )

    play_middle_of_track(device_id, min_track)

    sleep(pause_seconds)
    
    play_middle_of_track(device_id, max_track)

In [ ]:
df_apres_ski = pd.read_pickle("data/df_apres_ski.pkl")

In [ ]:
df_all_songs = (
    pd.read_pickle("data/all_songs_with_lyrics_cleaned.pkl")
    .assign(duration_minutes=lambda track: track["duration_ms"] / 1000 / 60)
)

# What do audio features this mean?!

In [ ]:
play_audio_feature_extremes(df_apres_ski, "acousticness")


In [ ]:
play_audio_feature_extremes(df_apres_ski, "danceability")

In [ ]:
# play_audio_feature_extremes(df_apres_ski, "energy")

In [ ]:
play_audio_feature_extremes(df_apres_ski, "liveness")

In [ ]:
play_audio_feature_extremes(df_apres_ski, "instrumentalness")
# here we see that the lyrics don't need to be complex
# and sometimes we don't even need words at all

In [ ]:
play_audio_feature_extremes(df_apres_ski, "speechiness")


In [ ]:
play_audio_feature_extremes(df_apres_ski, "valence")

# This kind of reflects what I was thinking when my NLP efforts didn't pay off

In [ ]:

track_series = df_apres_ski[df_apres_ski["id"] == "3S3Zrdg6WDjXTE8BfYGvE9"].iloc[0]
play_part_of_song(
    device_id=device_id,
    uri=track_series["uri"],
    start_position_ms=56000,
    n_seconds=10
)

# Then for a moment, I thought I was getting delusional

In [ ]:

# track_series = df_apres_ski[df_apres_ski["id"] == "5urUqiBMwHBsDOpzwLWubS"].iloc[0]
play_part_of_song(
    device_id=device_id,
    uri="https://open.spotify.com/track/5urUqiBMwHBsDOpzwLWubS",
    start_position_ms=57000,
    n_seconds=10
)

# https://open.spotify.com/track/5urUqiBMwHBsDOpzwLWubS?si=17f54d5f91b54869

## Based on this query I did a manual pre-selection to exclude songs you will need to learn outside of the office :-)

In [ ]:
# (
#     df_all_songs
#     .query("playlist == 'Après Ski'")
#     .query("popularity >= 40")
#     .query("language == 'German'") # This also excludes songs for which we don't have lyrics
# )

In [ ]:
# map song ids to starting point
pre_selection = {
    "5Qht2aUJcCjRuhrlHvvKt2": 84,
    "7KSOqRndT6D0d5Ok2yVrSh": 48,
    "1hDNg43vCjUHSJSzETUHdN": 50,
    "4HqlbB0BuahK05r6P2KcwP": 49,
    "1RcWWgnw5fVm3wcNq11zBu": 2,
    "6ksZJSo6h2OAlqQNQZEYIu": 51,
    "1PGSrlMKPm4FsgZvHtOY2Q": 27,
    "2A2PmKoiTzGbQRglDK0v2K": 45,
    "2kjlOZ10YPK3deMN45l4bS": 21,
    "2E68HPSmJxetiG8xjkNAJ3": 15,
    "7gJc9SPshSHnLSqz2d0ETV": 61,
    "65wKbNqjUTmdyk47Z31Czm": 111,
}


In [ ]:
df_selection = df_apres_ski[df_apres_ski["id"].isin(pre_selection.keys())]

In [ ]:
scores = {
    "acousticness":     {"minimize": 5, "maximize": 3},
    "danceability":     {"minimize": 0, "maximize": 10},
    "liveness":         {"minimize": 2, "maximize": 8},
    "speechiness":      {"minimize": 6, "maximize": 4},
    "valence":          {"minimize": 1, "maximize": 9},

}

In [ ]:
df_selection = df_selection[["id", *scores.keys()]]
selection_min = df_selection.min().to_dict()
selection_max = df_selection.max().to_dict()
selection_range = {feature: (selection_max[feature] - selection_min[feature]) for feature in scores.keys()}

In [ ]:
positions = {
    feature: votes["maximize"] / (votes["minimize"] + votes["maximize"]) 
    for feature, votes in scores.items()
}
required_levels = {
    feature: (selection_min[feature] + positions[feature] * selection_range[feature])
    for feature in scores.keys()
}


In [ ]:
required_levels

In [ ]:
required_levels["id"] = "required_levels"
df_required_levels = pd.DataFrame([required_levels], columns=df_selection.columns)

In [ ]:
selection_plus_required = pd.concat([
    df_selection,
    df_required_levels
    
], ignore_index=True)

In [ ]:
# Based on:
# https://stackoverflow.com/questions/20303323/distance-calculation-between-rows-in-pandas-dataframe-using-a-distance-matrix
distances = pdist(selection_plus_required[scores.keys()].values, metric='euclidean')
dist_matrix = squareform(distances)

In [ ]:
selection_plus_required["distance"] = dist_matrix[-1]

In [ ]:
winner_ids = (
    list(selection_plus_required
    [selection_plus_required["id"] != "required_levels"]
    .sort_values("distance", ascending=True)
    ["id"].values)
)

In [ ]:
winner_ids

In [ ]:

sp.user_playlist_replace_tracks(
    sp_user_id,
    playlist_id=playlist_id,
    tracks=winner_ids
    )

In [ ]:
# play top 3
for id in winner_ids[:3]:
    track_series = df_apres_ski[df_apres_ski["id"] == id].iloc[0]
    print(f'Playing {track_series["artist_names"]} - {track_series["name"]}')
    play_part_of_song(
        device_id=device_id,
        uri=track_series["uri"],
        start_position_ms=pre_selection[id] * 1000,
        n_seconds=10
    )

In [ ]:
import plotly.graph_objects as go
import numpy as np

In [ ]:
# x = list(range(1, len(scores.keys()) + 1))
x = list(scores.keys())
x_rev = x[::-1]

In [ ]:
# Based on: https://plotly.com/python/line-charts/

y_lower = [selection_min[feature] for feature in scores.keys()]
y_lower = y_lower[::-1]
y_upper = [selection_max[feature] for feature in scores.keys()]
y_mean = [required_levels[feature] for feature in scores.keys()]


gold_track = df_apres_ski[df_apres_ski["id"] == winner_ids[0]].iloc[0].to_dict()
y_gold_track = [gold_track[feature] for feature in scores.keys()]

silver_track = df_apres_ski[df_apres_ski["id"] == winner_ids[1]].iloc[0].to_dict()
y_silver_track = [silver_track[feature] for feature in scores.keys()]

bronze_track = df_apres_ski[df_apres_ski["id"] == winner_ids[2]].iloc[0].to_dict()
y_bronze_track = [bronze_track[feature] for feature in scores.keys()]


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=y_upper+y_lower,
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    # showlegend=False,
    name='Preselection',
))
fig.add_trace(go.Scatter(
    x=x, y=y_mean,
    line_color='rgb(0,100,80)',
    name='Wisdom of Crowd',
))

fig.add_trace(go.Scatter(
    x=x, y=y_gold_track,
    line_color='gold',
    name=gold_track["name"],
))

fig.add_trace(go.Scatter(
    x=x, y=y_silver_track,
    line_color='darkgrey',
    name=silver_track["name"],
))

fig.add_trace(go.Scatter(
    x=x, y=y_bronze_track,
    line_color='brown',
    name=bronze_track["name"],
))
# fig.add_trace(go.Scatter(
#     x=x, y=y3,
#     line_color='rgb(231,107,243)',
#     name='Ideal',
# ))

fig.update_traces(mode='lines')
fig.show()
